<a href="https://colab.research.google.com/github/zahraDehghanian97/classify_text/blob/master/word2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **prerequisit**

In [1]:
import pandas as pd
import numpy as np
from itertools import groupby
import re
from gensim.models import Word2Vec
from sklearn.metrics import balanced_accuracy_score

# **load dataset**

In [3]:
train_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/train_set.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/test_set.csv'
result_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/prediction2.csv'
model_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/word2vec.model'
stop_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/stop_word.csv'

In [4]:
# this function extract important word of each game
def prepare_string(string):
    result = []
    # remove html tags from string
    TAG_RE = re.compile(r'<[^>]+>')
    string = TAG_RE.sub('', string)
    # remove punctuation mark
    string = re.sub("[!()@.?؛:،-]",'', string)
    strings=string.split()
    # remove stop word
    for s in strings : 
      s = s.strip()
      if not s in stop_word.values and s!='' : 
        result.append(s)
    return result

# load short list of stop word 
stop_word = pd.read_csv(stop_path ,header = None, encoding = 'utf8')

# load train set
docs = []
id_docs = []
sentences = []
word_count = []
labels = []
train_data = pd.read_csv(train_path, encoding='utf8')
for d in train_data.values:
    id_docs.append(d[0])
    p_string = prepare_string(d[1])
    docs.append(p_string)
    sentences.extend(p_string)
    word_count.append(len(p_string))
    labels.append(d[2])
distinct_words = list(set(sentences))

# # split validation data
# id_docs_validation = id_docs[-150:]
# id_docs = id_docs[:-150]
# docs_validation = docs[-150:]
# docs = docs [:-150]
# word_count_validation = word_count[-150:]
# word_count = word_count[:-150]
# labels_validation = labels[-150:]
# labels = labels[:-150]


# load test set
docs_test = []
id_docs_test = []
test_data = pd.read_csv(test_path, encoding='utf8')
for d in test_data.values:
    id_docs_test.append(d[0])
    docs_test.append(prepare_string(d[1]))

docs_test = docs_test[1200:]
id_docs_test = id_docs_test[1200:]
print(docs[0])
print(id_docs_test[0])

['بازی', 'مین', 'برنامه', 'فکری', 'باید', 'مین', 'موجود', 'صفحه', 'بازی', 'کشف', 'قابلیت', 'برنامه', 'عبارتند', 'دارای', 'سطوح', 'دشواری', 'آسان', 'سخت', 'امکان', 'تعریف', 'بازی', 'سفارشی', 'نمایش', 'آمار', 'بازی', 'عملکرد', 'ذخیره', 'بازی', 'ناتمام', 'میتوانید', 'آینده', 'قسمت', 'بازی', 'ذخیره', 'مراجعه', 'بازی', 'ادامه', 'دهید', 'سایر', 'برنامه', 'بازی', 'متنوع', 'نیز', 'دیدن', 'سپاسگزارم']
40000


# **word2Vec Model**

In [ ]:
# this func creates word2VecModel of given docs
def create_word2Vec_model(docs, len_model):
    print("start creating word2Vec model with length word = " + str(len_model))
    model = Word2Vec(docs, window=3, min_count=1, workers=8, size=len_model)
    model.save(model_path+str(len_model))
    print("model creation finished")
    return


# this function loads the implemented word2vec model
def loadModel(len_model):
    print("start loading models...")
    return Word2Vec.load(model_path+str(len_model))


# this func is used to evaluate similarity between a query and all docs and return sorted docs
def findSimilar(query_terms, model):
    results = []
    counter=0
    new_query = []
    for q in query_terms :
        if q in model.wv.vocab : new_query.append(q)
    for doc in id_docs:
        new_doc = []
        for word in docs[counter]:
            if word in model.wv.vocab : new_doc.append(word)
        similarity = model.wv.n_similarity(new_query,new_doc)
        results.append((doc, similarity))
        counter+=1
    return sorted(results, key=lambda t: t[1], reverse=True)


# this func evaluates all given queries with given precisions (@k)
def evaluate_queries(queries, model,validation):
    labels_prediction = []
    temp = 0
    for query in queries:
        res = findSimilar(query, model)
        result = []
        for r in res[:10]:
            result.append(labels[r[0]])
        group = groupby(result)
        final_tag =  max(group, key=lambda k: len(list(k[1])))
        labels_prediction.append(final_tag[0])
        # if validation :
        #   gold = labels_validation[temp]
        #   print("result word2vec retrieval : " + str(final_tag[0])+"   gold result : " + str(gold))
        # else :
        print("evaluate result "+str(id_docs_test[temp])+"  predicted label : "+str(labels_prediction[-1]))
        dist_save=20
        if temp% dist_save == 0 :
          if (temp ==0):
            data =  { 'id' : id_docs_test[0] ,'label' : labels_prediction[:] }
            df = pd.DataFrame(data)
            df.to_csv(result_path, index=False)
          else :
            data =  { 'id' : id_docs_test[temp-dist_save+1:temp+1] ,'label' : labels_prediction[temp-dist_save+1 :] }
            df = pd.DataFrame(data)
            df.to_csv(result_path, mode='a',header = None,index=False)
            print(">>>> save result from the begining to data number "+str(id_docs_test[temp]))
        temp += 1
    print (labels_prediction)
    return labels_prediction

len_model = 200

# # create Model
# create_word2Vec_model(docs, len_model)

# load model
model = loadModel(len_model)
print("model with lentgh word = " + str(len_model))

# evaluation
validation = False
labels_prediction = evaluate_queries(docs_test, model,validation)
print("evaluation finished... at last !")



start loading models...
model with lentgh word = 200
evaluate result 41200  predicted label : 4
evaluate result 41201  predicted label : 4
evaluate result 41202  predicted label : 1
evaluate result 41203  predicted label : 4
evaluate result 41204  predicted label : 1
evaluate result 41205  predicted label : 6
evaluate result 41206  predicted label : 7
evaluate result 41207  predicted label : 3
evaluate result 41208  predicted label : 3
evaluate result 41209  predicted label : 6
evaluate result 41210  predicted label : 5
evaluate result 41211  predicted label : 3
evaluate result 41212  predicted label : 5
evaluate result 41213  predicted label : 5
evaluate result 41214  predicted label : 8
evaluate result 41215  predicted label : 2
evaluate result 41216  predicted label : 4
evaluate result 41217  predicted label : 4
evaluate result 41218  predicted label : 0
evaluate result 41219  predicted label : 3
evaluate result 41220  predicted label : 4
>>>> save result from the begining to data n